In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load and Clean Data
file_path = "/content/FINAL_ESS_EUROSTAT_DATA.csv"
df = pd.read_csv(file_path)

# Convert column names to lowercase and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(" ", "_")

# Drop 'Unnamed:_0' if it exists in the dataset
if "unnamed:_0" in df.columns:
    df.drop(columns=["unnamed:_0"], inplace=True)

# Rename columns
df.rename(columns={'hinctnta': 'household_income', 'chldhm': 'have_child', 'gdp': 'gdppc'}, inplace=True)

# Drop unnecessary columns
drop_cols = ['far_right_support', 'cultural_impact', 'total_pop', 'rural_urban', 'pop_change']
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

# Step 2: Reverse Coding for Specific Variables
reverse_cols = ["immigration_economy", "perceived_economic_competition", "jobs_taken"]
for col in reverse_cols:
    df[col] = df[col].apply(lambda x: 10 - x if pd.notna(x) else x)

# Step 3: Identify Numerical and Categorical Variables
exclude_cols = ["country", "nuts2", "idno", "year", "nuts1", "eisced", "immig_background", "gndr"]  # Exclude 'gndr' from numerical processing
numerical_cols = [col for col in df.columns if col not in exclude_cols]

# Ensure categorical variables are included (e.g., Gender)
categorical_cols = ["gndr"]  # Add more categorical variables if needed
df["gndr"] = df["gndr"].map({1: 0, 2: 1}).dropna()  # Remove unknown gender cases

# Step 4: Identify Eurostat Data (These Should Not Be Standardized)
eurostat_vars = ["gdppc", "pop_density", "unemployment", "net_mig"]
standardize_cols = [col for col in numerical_cols if col not in eurostat_vars]

# Apply Min-Max Scaling (0-1 normalization) **only to non-Eurostat data**
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[standardize_cols] = scaler.fit_transform(df[standardize_cols])  # Eurostat data remains unchanged
df_scaled = df_scaled[~df_scaled["country"].isin(["DE", "UK"])]

# Step 5: Categorize Education Levels
df_scaled["education_numeric"] = df_scaled["eisced"].apply(lambda x:
    2 if x <= 2 else
    3 if x == 3 else
    4 if x == 4 else
    5 if x >= 5 else None)

# Step 6: Compute **Education Level Statistics**
edu_stats_corrected = df_scaled[df_scaled["education_numeric"].notna()].groupby("education_numeric")[standardize_cols].agg(["mean", "std", "min", "max"]).T

# Reset index for clarity
edu_stats_corrected = edu_stats_corrected.reset_index()
edu_stats_corrected.rename(columns={"level_0": "Variable", "level_1": "Statistic"}, inplace=True)

# Round values to three decimal places
edu_stats_corrected.iloc[:, 2:] = edu_stats_corrected.iloc[:, 2:].round(3)

# Step 7: Compute General Descriptive Statistics (Without Standardizing Eurostat Data)
desc_vars = df_scaled[numerical_cols]
desc_stats = desc_vars.describe().T[['count', 'mean', 'std', 'min', 'max']]
desc_stats.columns = ["N", "Mean", "SD", "Min", "Max"]
desc_stats = desc_stats.round(2)

# Compute categorical variable counts (e.g., Gender)
gender_stats = df_scaled["gndr"].describe().T.to_frame().T[["count", "mean", "std", "min", "max"]]
gender_stats.columns = ["N", "Mean", "SD", "Min", "Max"]
gender_stats.insert(0, "Variable", "Gender")
gender_stats[["N", "Mean", "SD", "Min", "Max"]] = gender_stats[["N", "Mean", "SD", "Min", "Max"]].round(2)
# Step 8: Format Descriptive Statistics Table
dv_label = pd.DataFrame({"Variable": ["Dependent Variable"], "N": [""], "Mean": [""], "SD": [""], "Min": [""], "Max": [""]})
dv_row = pd.DataFrame({"Variable": ["Outgroup Hostility"], **desc_stats.loc[["imwbcnt"]].reset_index(drop=True).to_dict(orient="list")})

iv_label = pd.DataFrame({"Variable": ["Independent Variables"], "N": [""], "Mean": [""], "SD": [""], "Min": [""], "Max": [""]})
iv_rows = desc_stats.drop(index=["imwbcnt"] + eurostat_vars, errors='ignore').reset_index().rename(columns={"index": "Variable"})

# Step 9: Compute **Education Level Statistics**
edu_label = pd.DataFrame({
    "Variable": ["Educational Level",
                 "Low (≤ ISCED 2)",
                 "Medium (ISCED 3)",
                 "Medium (ISCED 4)",
                 "High (≥ ISCED 5)"],
    "N": ["",
          df_scaled[df_scaled["education_numeric"] == 2].shape[0],
          df_scaled[df_scaled["education_numeric"] == 3].shape[0],
          df_scaled[df_scaled["education_numeric"] == 4].shape[0],
          df_scaled[df_scaled["education_numeric"] == 5].shape[0]],
    "Mean": ["",
             round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "mean", 2.0].mean(), 2),
             round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "mean", 3.0].mean(), 2),
             round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "mean", 4.0].mean(), 2),
             round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "mean", 5.0].mean(), 2)],
    "SD": ["",
           round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "std", 2.0].mean(), 2),
           round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "std", 3.0].mean(), 2),
           round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "std", 4.0].mean(), 2),
           round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "std", 5.0].mean(), 2)],
    "Min": ["",
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "min", 2.0].min(), 2),
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "min", 3.0].min(), 2),
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "min", 4.0].min(), 2),
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "min", 5.0].min(), 2)],
    "Max": ["",
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "max", 2.0].max(), 2),
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "max", 3.0].max(), 2),
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "max", 4.0].max(), 2),
            round(edu_stats_corrected.loc[edu_stats_corrected["Statistic"] == "max", 5.0].max(), 2)]
})

# Step 10: Append **Education Levels First**, Then Add Eurostat Data & Categorical Data**
desc_stats_final = pd.concat([dv_label, dv_row, iv_label, iv_rows, gender_stats, edu_label], ignore_index=True)

# Step 11: Append **Eurostat Data Last to Prevent Duplication**
eurostat_rows = desc_stats.loc[eurostat_vars].reset_index().rename(columns={"index": "Variable"})
desc_stats_final = pd.concat([desc_stats_final, eurostat_rows], ignore_index=True)

# Step 12: Check for Aggregation Errors
aggregation_check = edu_stats_corrected.isna().sum().sum()

# Step 13: Rename Variables for Readability
variable_labels = {
    "imwbcnt": "Outgroup Hostility",
    "gdppc": "GDP per Capita",
    "pop_density": "Population Density",
    "unemployment": "Unemployment Rate",
    "net_mig": "Crude Net Migration Rate",
    "immigration_economy": "Economic Impact of Immigration",
    "perceived_economic_competition": "Perceived Economic Competition",
    "jobs_taken": "Immigration’s Job Impact",
    "education_numeric": "Education Level",
    "have_child": "Parental Status",
    "lrscale": "Political Ideology",
    "household_income": "Household Total Net Income",
    "social_contact": "Social Contact",
    "age": "Age"
}

# Apply renaming to the descriptive statistics table
desc_stats_final["Variable"] = desc_stats_final["Variable"].replace(variable_labels)
print(desc_stats_final)
standardized_file_path = "/content/ESS_EUROSTAT_final_standardized.csv"
df_scaled.to_csv(standardized_file_path, index=False)


                          Variable        N      Mean        SD     Min  \
0               Dependent Variable                                        
1               Outgroup Hostility  65010.0      0.49      0.22     0.0   
2            Independent Variables                                        
3                   Social Contact  69711.0      0.69      0.36     0.0   
4   Economic Impact of Immigration  66855.0      0.51      0.24     0.0   
5   Perceived Economic Competition  65797.0      0.56      0.22     0.0   
6         Immigration’s Job Impact  67275.0      0.54      0.23     0.0   
7                minority_presence  67364.0      0.31      0.34     0.0   
8               Political Ideology  60532.0      0.51      0.22     0.0   
9                  Parental Status  67839.0      0.08      0.08     0.0   
10                             Age  67844.0      0.04      0.07     0.0   
11      Household Total Net Income  54321.0      0.51      0.29     0.0   
12                       

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
file_path = "/content/FINAL_ESS_EUROSTAT_DATA.csv"  # Adjust path if necessary
df = pd.read_csv(file_path)

# Convert column names to lowercase and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(" ", "_")

# Drop 'Unnamed:_0' if it exists in the dataset
if "unnamed:_0" in df.columns:
    df.drop(columns=["unnamed:_0"], inplace=True)
if 'cultural_impact' in df.columns:
    df.drop(columns=['cultural_impact'], inplace=True)

if 'total_pop' in df.columns:
    df.drop(columns=['total_pop'], inplace=True)
# Rename specific columns
df.rename(columns={'hinctnta': 'household_income', 'chldhm': 'have_child', 'gdp': 'gdppc'}, inplace=True)

# Drop 'far_right_support' if it exists in the dataset
if 'far_right_support' in df.columns:
    df.drop(columns=['far_right_support'], inplace=True)

# Reverse coding for specific variables
reverse_cols = ["cultural_impact", "immigration_economy", "perceived_economic_competition", "jobs_taken"]
for col in reverse_cols:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 10 - x if pd.notna(x) else x)

# Identify numerical and categorical variables
exclude_cols = ["country", "nuts2", "idno", "year", "gndr", "nuts1", "eisced", "immig_background"]
numerical_cols = [col for col in df.columns if col not in exclude_cols]

# Identify Eurostat Data (These Should Not Be Standardized)
eurostat_vars = ["gdppc", "pop_change", "pop_density", "unemployment", "net_mig"]
standardize_cols = [col for col in numerical_cols if col not in eurostat_vars]

# Apply Min-Max Scaling (0-1 normalization) only to non-Eurostat data
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[standardize_cols] = scaler.fit_transform(df[standardize_cols])  # Eurostat data remains unchanged

# Categorize Education Levels (Grouping eisced <= 2)
df_scaled["education_numeric"] = df_scaled["eisced"].apply(lambda x:
    2 if x <= 2 else  # Group all eisced <= 2 together
    3 if x == 3 else
    4 if x == 4 else
    5 if x >= 5 else None)  # Retain None if eisced is missing

df_scaled
# Save the standardized dataset to CSV
#standardized_file_path = "/content/ESS_EUROSTAT_standardized.csv"
#df_scaled.to_csv(standardized_file_path, index=False)

# Return the file path
# standardized_file_path


,country,year,nuts2,idno,social_contact,immigration_economy,perceived_economic_competition,jobs_taken,rural_urban,minority_presence,...,eisced,household_income,immig_background,nuts1,gdppc,pop_change,pop_density,unemployment,net_mig,education_numeric
0,BE,2002,BE10,56,0.0,0.7,0.6,0.6,0.25,0.0,...,2.0,0.444444,0.0,BE1,NaN,978384.0,6119.3,15.9,10.2,2.0
1,DE,2003,DEF,101429,1.0,0.5,0.5,0.7,0.25,0.0,...,2.0,0.555556,0.0,DEF,21500.0,2816507.0,178.9,8.6,NaN,2.0
2,DE,2003,DEB,164508,0.0,0.5,0.5,0.5,0.25,1.0,...,3.0,0.444444,0.0,DEB,21800.0,4057727.0,204.5,6.3,NaN,3.0
3,DE,2003,DEB,164520,0.0,0.0,0.5,0.0,0.00,1.0,...,2.0,0.444444,0.0,DEB,21800.0,4057727.0,204.5,6.3,NaN,2.0
4,DE,2003,DEB,165403,1.0,0.5,0.7,0.5,0.75,0.0,...,3.0,0.666667,0.0,DEB,21800.0,4057727.0,204.5,6.3,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75345,GB,2014,UKM,100002832,1.0,0.5,NaN,0.5,0.75,0.0,...,1.0,NaN,0.0,UKM,NaN,NaN,NaN,NaN,NaN,2.0
75346,GB,2003,UKM,101214,1.0,0.5,0.8,0.3,0.25,0.5,...,0.0,0.555556,0.0,UKM,NaN,NaN,NaN,NaN,NaN,2.0
75347,GB,2002,UKM,101220,1.0,0.5,0.8,0.7,0.25,0.5,...,0.0,0.444444,0.0,UKM,NaN,NaN,NaN,NaN,NaN,2.0
75348,GB,2002,UKM,101702,1.0,0.5,0.8,0.5,0.50,0.0,...,0.0,0.111111,0.0,UKM,NaN,NaN,NaN,NaN,NaN,2.0


In [ ]:
import matplotlib.pyplot as plt

def dataframe_to_image_adjusted(df, title, filename):
    fig, ax = plt.subplots(figsize=(14, min(0.4 * len(df), 16)))  # Adjust size dynamically
    ax.axis("tight")
    ax.axis("off")

    # Create a structured table with professional formatting
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc="center", loc="center",
                     colColours=["#D3D3D3"]*df.shape[1], cellColours=[["#FFFFFF"]*df.shape[1]]*len(df), edges="horizontal")

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)  # Scale table for better visibility

    # Adjust column widths: Increase "Variable" column width, other columns narrower
    column_widths = [0.3] + [0.1] * (len(df.columns) - 1)  # "Variable" gets more space

    for j, width in enumerate(column_widths):
        table.auto_set_column_width([j])  # Apply width adjustment
        for i in range(len(df) + 1):  # +1 for header row
            table[i, j].set_width(width)

    # Style header row separately
    for (i, j), cell in table.get_celld().items():
        if i == 0:  # Header row styling
            cell.set_fontsize(11)
            cell.set_text_props(weight="bold")
            cell.set_edgecolor("black")  # Dark border for better separation
            cell.set_facecolor("#D3D3D3")  # Light grey background
        else:
            cell.set_edgecolor("black")  # Light horizontal grid lines

    # Add a professional-style title
    plt.title(title, fontsize=14, fontweight="bold", pad=20, color="black")

    # Save the figure
    plt.savefig(filename, bbox_inches="tight", dpi=300)
    plt.close()

# Example usage
desc_stats_img = "descriptive_statistics_adjusted.png"
dataframe_to_image_adjusted(desc_stats_final, "Descriptive Statistics Table", desc_stats_img)
